In [1]:
!pwd

/Recommendation/GRU4REC


In [3]:
import numpy as np
import pandas as pd
import datetime
import os

In [4]:
dataBefore = '../YooChoose/yoochoose-clicks.dat'
dataTestBefore = '../YooChoose/yoochoose-test.dat'
dataAfter = '../YooChoose/'
dayTime = 86400

In [7]:
def removeShortSessions(data):
    # delete sessions of length < 1
    sessionLen = data.groupby('SessionID').size()
    data = data[np.in1d(data.SessionID, sessionLen[sessionLen >1].index)]
    return data

In [8]:
# Read Dataset in pandas Dataframe (Ignore Category Column)
train = pd.read_csv(dataBefore, sep=',', header = None, usecols = [0, 1, 2], dtype = {0:np.int32, 1:str, 2:np.int64})
test = pd.read_csv(dataTestBefore, sep=',', header = None, usecols = [0, 1, 2], dtype = {0:np.int32, 1:str, 2:np.int64})
train.columns = ['SessionID', 'Time', 'ItemID']
test.columns = ['SessionID', 'Time', 'ItemID']
train['Time'] = train.Time.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp())
test['Time'] = test.Time.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fz').timestamp())

In [10]:
train = removeShortSessions(train)
itemLen = train.groupby('ItemID').size()
train = train[np.in1d(train.ItemID, itemLen[itemLen > 4].index)]
train = removeShortSessions(train)

In [15]:
test = test[np.in1d(test.ItemID, train.ItemID)]
test = removeShortSessions(test)

print('Testing Set has', len(test), 'Events, ', test.SessionID.nunique(), 'Sessions, and', train.ItemID.nunique(), 'Items\n\n')
test.to_csv(dataAfter +'recSys15Test.txt', sep=',', index=False)

Testing Set has 7924883 Events,  1994942 Sessions, and 37301 Items




In [20]:
timeMax = train.Time.max()
sessionMaxTime = train.groupby('SessionID').Time.max()
sessionTrain = sessionMaxTime[sessionMaxTime < (timeMax - dayTime)].index
sessionValid = sessionMaxTime[sessionMaxTime >= (timeMax -dayTime)].index
trainTR = train[np.in1d(train.SessionID, sessionTrain)]
trainVD = train[np.in1d(train.SessionID, sessionValid)]
trainVD = trainVD[np.in1d(trainVD.ItemID, trainTR.ItemID)]
trainVD = removeShortSessions(trainVD)

print('Training Set has', len(trainTR), 'Events, ', trainTR.SessionID.nunique(), 'Sessions, and', trainTR.ItemID.nunique(), 'Items\n\n')
trainTR.to_csv(dataAfter + 'recSys15TrainOnly.txt', sep = ',', index = False)
print('Validation Set has', len(trainVD), 'Events, ', trainVD.SessionID.nunique(), 'Sessions, and' , trainVD.ItemID.nunique(), 'Items\n\n')
trainVD.to_csv(dataAfter + 'recSys15Valid.txt', sep = ',', index = False)

Training Set has 31636405 Events,  7966041 Sessions, and 37298 Items


Validation Set has 71219 Events,  15323 Sessions, and 6749 Items




In [32]:
trainVD

,SessionID,Time,ItemID
32186847,11265009,1.412004e+09,214586805
32186848,11265009,1.412004e+09,214509260
32186868,11265017,1.412018e+09,214857547
32186869,11265017,1.412018e+09,214857268
32186870,11265017,1.412018e+09,214857260
...,...,...,...
33003915,11299816,1.412021e+09,214859859
33003916,11299816,1.412021e+09,214859859
33003917,11299816,1.412022e+09,214859859
33003918,11299816,1.412023e+09,214746399


In [34]:
trainTR

,SessionID,Time,ItemID
0,1,1.396868e+09,214536502
1,1,1.396868e+09,214536500
2,1,1.396868e+09,214536506
3,1,1.396868e+09,214577561
4,2,1.396879e+09,214662742
...,...,...,...
33003939,11299809,1.411638e+09,214819412
33003940,11299809,1.411638e+09,214830939
33003941,11299811,1.411585e+09,214854855
33003942,11299811,1.411585e+09,214854838
